In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 30

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv01 = nn.Sequential(         
            nn.Conv2d(64, 64*2, 1, 1, padding = "same"),
            nn.BatchNorm2d(64*2),
            nn.ReLU(),
            nn.Conv2d(64*2, 64*2, 3, 1, padding = "same"),
            nn.BatchNorm2d(64*2),
            nn.ReLU(),
            nn.Conv2d(64*2, 64, 1, 1, padding = "same"),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.conv02 = nn.Sequential(         
            nn.Conv2d(32, 32*4, 1, 1, padding = "same"),
            nn.BatchNorm2d(32*4),
            nn.ReLU(),
            nn.Conv2d(32*4, 32*4, 3, 1, padding = "same"),
            nn.BatchNorm2d(32*4),
            nn.ReLU(),
            nn.Conv2d(32*4, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2) 
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        identity = x
        x = self.conv01(x)
        x = identity + x
        x = self.conv3(x)
        identity = x
        x = self.conv02(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

CNN_model(
  (conv01): Sequential(
    (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (conv02): Sequential(
    (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(128, 32, kernel_size

# Step 3 Pass in the model to start training and testing

In [6]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/30]	|	Step: [9/27]	|	Loss: 3.0126
Epoch: [1/30]	|	Step: [18/27]	|	Loss: 3.0697
Epoch: [1/30]	|	Step: [27/27]	|	Loss: 3.0936
9.393579483032227
epoch: 01 | Accuracy: 9.39 Loss: 82.1798

Epoch: [2/30]	|	Step: [9/27]	|	Loss: 2.6790
Epoch: [2/30]	|	Step: [18/27]	|	Loss: 2.3565
Epoch: [2/30]	|	Step: [27/27]	|	Loss: 2.3784
24.13793182373047
epoch: 02 | Accuracy: 24.14 Loss: 71.4511

Epoch: [3/30]	|	Step: [9/27]	|	Loss: 1.8624
Epoch: [3/30]	|	Step: [18/27]	|	Loss: 2.0459
Epoch: [3/30]	|	Step: [27/27]	|	Loss: 1.9326
35.79072570800781
epoch: 03 | Accuracy: 35.79 Loss: 57.9977

Epoch: [4/30]	|	Step: [9/27]	|	Loss: 1.6338
Epoch: [4/30]	|	Step: [18/27]	|	Loss: 1.3800
Epoch: [4/30]	|	Step: [27/27]	|	Loss: 1.7272
54.10226058959961
epoch: 04 | Accuracy: 54.10 Loss: 40.2859

Epoch: [5/30]	|	Step: [9/27]	|	Loss: 1.1400
Epoch: [5/30]	|	Step: [18/27]	|	Loss: 0.8620
Epoch: [5/30]	|	Step: [27/27]	|	Loss: 1.0078
70.74910736083984
epoch: 05 | Accuracy: 70.75 Loss: 26.1802

Epoch: [6/30]	|	Step: [9/2

In [7]:
print(Accy_list)

[9.393579483032227, 24.13793182373047, 35.79072570800781, 54.10226058959961, 70.74910736083984, 77.5267562866211, 85.96908569335938, 92.98454284667969, 96.90843963623047, 98.09750366210938, 97.62187957763672, 97.2651596069336, 98.2164077758789, 99.40547180175781, 99.64328002929688, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [8]:
print(Loss_list)

[82.17976999282837, 71.45108962059021, 57.99769699573517, 40.285903453826904, 26.18021321296692, 18.026127696037292, 12.627059683203697, 7.45903217792511, 3.7889652512967587, 2.6091026477515697, 2.6261570639908314, 2.7860855627804995, 2.264484351500869, 1.3382200319319963, 0.6626178352162242, 0.253040318377316, 0.13710952596738935, 0.10151268704794347, 0.07264475547708571, 0.06972628855146468, 0.06195577880134806, 0.07492699008435011, 0.07547929242718965, 0.05857550841756165, 0.045172624464612454, 0.04098108096513897, 0.03624580753967166, 0.033753263764083385, 0.028516276186564937, 0.028794323152396828]


In [9]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 90.73 %, Loss: 11.7398 
